## EE 242 Lab 2 – Convolution

Author names go here 

(This should be a markup cell, which means that when you run it you just get formated text.)

In [11]:
# We'll refer to this as the "import cell." Every module you import should be imported here.
%matplotlib notebook
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# import whatever other modules you use in this lab -- there are more that you need than we've included 

## Summary

In this lab, you will work through a series of exercises involving convolution of time signals and images.  This is a two-week lab.  You should plan on completing the first 3 assignments in the first week.

## Lab 2 turn in checklist

•	Pre-lab (upload to canvas before lab)

•	Lab 1 Jupyter notebook with code for the first 4 exercises assignment in separate cells. Each assignment cell should contain markdown cells (same as lab overview cells) for the responses to lab report questions. Include your lab members’ names at the top of the notebook.

•	1 individual Jupyter notebook with code + markdown cells for the last exercise

Note: The pre-lab should be done individually, and all other assignments should be completed in groups of 3-4 people.


## Assignment 1 -- Simple Convolutions

In [12]:
# Assignment 1 - Time Scaling Function

import numpy as np
import matplotlib.pyplot as plt

# Part A - Create signals
x = np.zeros(13)
x[2:9] = 1  # Input signal x[n]

h1 = np.zeros(13)
h1[:4] = 1  # Impulse response h1[n]

h2 = np.zeros(13)
h2[0], h2[1] = 1, -1  # Impulse response h2[n]

# Part B - Convolve signals
y1 = np.convolve(x, h1, mode='full')  # Convolution of x[n] and h1[n]
y2 = np.convolve(x, h2, mode='full')  # Convolution of x[n] and h2[n]

# Part C - Plot signals
nx = np.arange(13)
ny1 = np.arange(len(y1))
ny2 = np.arange(len(y2))

plt.figure(figsize=(8, 6))
plt.subplot(3, 1, 1)
plt.stem(nx, x, 'b', markerfmt='bo', basefmt=' ')
plt.ylim(-2, 5)
plt.xlim(0, 20)
plt.title('x[n]')

plt.subplot(3, 1, 2)
plt.stem(ny1, y1, 'r', markerfmt='ro', basefmt=' ')
plt.ylim(-2, 5)
plt.xlim(0, 20)
plt.title('y1[n]')

plt.subplot(3, 1, 3)
plt.stem(ny2, y2, 'g', markerfmt='go', basefmt=' ')
plt.ylim(-2, 5)
plt.xlim(0, 20)
plt.title('y2[n]')

plt.tight_layout()
plt.show()



<IPython.core.display.Javascript object>

###  Discussion

The system corresponding to impulse response h1[n] captures the moving average of the input signal. The largest values of y1[n] correspond to regions where the input signal x[n] has a constant height of 1.

The system corresponding to impulse response h2[n] captures the differences between successive samples of the input signal. The largest values of y2[n] correspond to the edges of the box in the input signal x[n], where the signal changes rapidly.


## Assignment 2 -- Smoothing Signals

In [13]:
# Assignment 2 - Amplitude Operations on Signals

# Part A
srate = 1000
time = np.arange(0, 2, 1 / srate)
n = len(time)

p = 10
amp = 20
base = np.interp(np.linspace(0, p, n), np.arange(0, p), np.random.rand(p) * amp)

noiseamp = 2
noise = noiseamp * np.random.randn(n)

noisy_signal = base + noise

plt.figure(figsize=(8, 6))
plt.subplot(2, 1, 1)
plt.plot(time, base)
plt.ylim(0, 25)
plt.title('Original Signal')
plt.xlabel('Time (s)')

plt.subplot(2, 1, 2)
plt.plot(time, noisy_signal)
plt.ylim(0, 25)
plt.title('Noisy Signal')
plt.xlabel('Time (s)')

plt.tight_layout()
plt.show()

# Part B
k = 20
filtsig1 = np.zeros(n)

for i in range(k, n - k):
    filtsig1[i] = np.mean(noisy_signal[i - k:i + k + 1])

plt.figure(figsize=(8, 6))
plt.plot(time, noisy_signal, label='Noisy Signal')
plt.plot(time, base, label='Original Signal')
plt.plot(time, filtsig1, label='Filtered Signal (Running Mean)')
plt.ylim(0, 25)
plt.title('Filtered Signal (Running Mean)')
plt.xlabel('Time (s)')
plt.legend()
plt.show()

# Part C
hfilt = np.ones(2 * k + 1) / (2 * k + 1)
filtsig2 = np.convolve(noisy_signal, hfilt, mode='same')

plt.figure(figsize=(8, 6))
plt.plot(time, filtsig1, label='Filtered Signal 1 (Running Mean)')
plt.plot(time, filtsig2, label='Filtered Signal 2 (Convolution)')
plt.ylim(0, 25)
plt.title('Comparison of Filtered Signals')
plt.xlabel('Time (s)')
plt.legend()
plt.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

###  Discussion

Both methods, the running mean and the convolution with a box filter, smooth the noisy signal. The running mean filter calculates the average value of +/- k samples. The convolution with the box filter also calculates a weighted average but uses the convolution function.

The differences between the two methods are minimal, and both effectively smooth the signal. When the noise is amplified, both methods still smooth the signal, but the result will be less accurate. When the value of k is changed, the smoothing effect will change as well. For larger values of k, the smoothing will be stronger, and for smaller values of k, the smoothing will be weaker.

In terms of system properties, both methods act as low-pass filters, removing high-frequency noise while preserving the low-frequency components of the signal.


## Assignment 3 -- Removing Noise from an Audio Signal 

In [14]:
# Assignment 3 -- Title of assignment 3

import simpleaudio as sa
from scipy.io import wavfile

# Part A
fs, tr_orig = wavfile.read('trombone11.wav')
noise = 100 * np.random.randn(len(tr_orig))
tr_noisy = tr_orig + noise
wavfile.write('tr_noisy.wav', fs, tr_noisy.astype(np.int16))

# Part B
k = 20
hfilt = np.ones(2 * k + 1) / (2 * k + 1)
tr_filt = np.convolve(tr_noisy, hfilt, mode='same')
wavfile.write('tr_filt.wav', fs, tr_filt.astype(np.int16))

# Part C
wavfile.write('tr_orig.wav', fs, tr_orig.astype(np.int16))
tr_orig_audio = sa.WaveObject.from_wave_file('tr_orig.wav')
tr_noisy_audio = sa.WaveObject.from_wave_file('tr_noisy.wav')
tr_filt_audio = sa.WaveObject.from_wave_file('tr_filt.wav')

print("Playing Original Audio")
tr_orig_audio.play().wait_done()
print("Playing Noisy Audio")
tr_noisy_audio.play().wait_done()
print("Playing Filtered Audio")
tr_filt_audio.play().wait_done()



Playing Original Audio
Playing Noisy Audio
Playing Filtered Audio


###  Discussion

The original audio signal is clear and crisp. After adding noise, the noisy audio signal sounds distorted and is harder to discern. After applying the convolution filter to remove the noise, the filtered audio signal sounds cleaner than the noisy one but not as clear as the original. Some of the noise is still present, and the signal may sound slightly muffled.

When the value of k is increased, the smoothing effect becomes stronger, which results in better noise removal. However, this also leads to a more muffled sound, as the high-frequency components of the original signal are attenuated. When the value of k is decreased, the smoothing effect is weaker, resulting in less noise removal, and the filtered audio signal will still have a significant amount of noise. Balancing the value of k is crucial to achieving the best noise removal without losing too much of the original signal quality.


## Assignment 4 -- Convolution with Images

In [15]:
# Import necessary libraries
from skimage import color
from scipy import ndimage

# Part A
def sobel_edge_detector(image):
    sobel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    sobel_y = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])
    
    horizontal_edges = ndimage.convolve(image, sobel_x)
    vertical_edges = ndimage.convolve(image, sobel_y)
    combined_edges = np.hypot(horizontal_edges, vertical_edges)
    
    return horizontal_edges, vertical_edges, combined_edges

# Part B
image = plt.imread('dragonfly.jpg')
gray_image = color.rgb2gray(image)

horizontal_edges, vertical_edges, combined_edges = sobel_edge_detector(gray_image)

fig, axes = plt.subplots(2, 2, figsize=(10, 10))
axes[0, 0].imshow(gray_image, cmap='gray')
axes[0, 0].set_title("Original Grayscale Image")
axes[0, 1].imshow(horizontal_edges, cmap='gray')
axes[0, 1].set_title("Horizontal Edges")
axes[1, 0].imshow(vertical_edges, cmap='gray')
axes[1, 0].set_title("Vertical Edges")
axes[1, 1].imshow(combined_edges, cmap='gray')
axes[1, 1].set_title("Combined Edges")
plt.show()

# Part C
smooth_kernel = np.full((10, 10), 0.01)
smooth_image = ndimage.convolve(gray_image, smooth_kernel)

fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(gray_image, cmap='gray')
axes[0].set_title("Original Grayscale Image")
axes[1].imshow(smooth_image, cmap='gray')
axes[1].set_title("Smoothed Image")
plt.show()

# Part D
smooth_horizontal_edges, smooth_vertical_edges, smooth_combined_edges = sobel_edge_detector(smooth_image)

fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(combined_edges, cmap='gray')
axes[0].set_title("Edges from Original Image")
axes[1].imshow(smooth_combined_edges, cmap='gray')
axes[1].set_title("Edges from Smoothed Image")
plt.show()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Discussion

Describe the differences in the results using the edge detector or the original and smoothed image. Comment on how the results change if you use a larger size smoothing filter.

This code defines a function sobel_edge_detector() in Part A, which takes an image as input and returns the horizontal edges, vertical edges, and combined edges using the Sobel edge detector. In Part B, it reads the dragonfly image, converts it to grayscale, and runs the edge detector function, then displays the original image and the outputs in a 2x2 figure. In Part C, it defines a 10x10 smoothing kernel, convolves the image with the kernel, and plots the original and smoothed images side-by-side. In Part D, it runs the edge detector function on the smoothed dragonfly image and plots the result side-by-side with the result from the edge detector on the original image.